# モデル作成

## インポート

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Conv1D, MaxPooling1D, Flatten, GlobalAveragePooling1D, Reshape

from lib.MassBankRecord import MassBankRecord

ModuleNotFoundError: No module named 'sklearn'